# Comparing VBD Outputs: Waymax vs GPUDrive

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import mediapy
import os
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import torch
# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

# GPUDrive dependencies
import gpudrive
from gpudrive.env.config import EnvConfig, RenderConfig, SceneConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.env.dataset import SceneDataLoader
from gpudrive.visualize.utils import img_from_fig
from gpudrive.integrations.vbd.sim_agent.sim_actor import VBDTest

# Plotting
sns.set("notebook")
sns.set_style("ticks", rc={"figure.facecolor": "none", "axes.facecolor": "none"})
#%config InlineBackend.figure_format = 'svg'

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-04-15 14:47:51.138446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744742871.227262   10057 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744742871.257112   10057 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744742871.450339   10057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744742871.450399   10057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744742871.450403   10057 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

### Configurations

In [2]:
DATA_DIR = 'data/processed' # Base data path
CKPT_PATH = 'gpudrive/integrations/vbd/weights/epoch=18.ckpt'

SCENARIO_ID = 'efc5cbe01b4a526f'

FPS = 20
INIT_STEPS = 11 # Warmup period
MAX_CONTROLLED_OBJECTS = 32

In [3]:
# Load VBD model
def load_vbd_model(model_path, device="cpu"):
    """Load the VBD model from a checkpoint, forcing CPU usage."""
    # Force the model to be loaded on CPU
    model = VBDTest.load_from_checkpoint(model_path, torch.device(device))
    # Make sure model is in eval mode
    _ = model.eval()
    return model

vbd_model = load_vbd_model(CKPT_PATH, device="cpu")

### Make Videos

In [4]:
#Init GPUDrive env
env_config = EnvConfig(
    init_steps=INIT_STEPS, # Warmup period
    dynamics_model="state", # Use state-based dynamics model
    dist_to_goal_threshold=1e-5, # Trick to make sure the agents don't disappear when they reach the goal
    init_mode = 'all_non_trivial',
    max_controlled_agents=32,
    goal_behavior='ignore'
)
        
# Make env
gpudrive_env = GPUDriveTorchEnv(
    config=env_config,
    data_loader = SceneDataLoader(
        root="data/processed/training",
        batch_size=2,
        dataset_size=2,
    ),
    render_config=RenderConfig(resolution=(400, 400)),
    max_cont_agents=MAX_CONTROLLED_OBJECTS, # Maximum number of agents to control per scene
    device="cpu",
)
gpudrive_sample_batch = gpudrive_env._generate_sample_batch()
predictions = vbd_model.sample_denoiser(gpudrive_sample_batch)
vbd_output = predictions["denoised_trajs"].to("cpu").detach()

# Reset predictions tensor
pred_trajs = torch.zeros((gpudrive_env.num_worlds, gpudrive_env.max_agent_count, env_config.episode_len-INIT_STEPS, 10))

# World means
world_means = gpudrive_env.sim.world_means_tensor().to_torch()[:, :2].to("cpu")

# Fill pred_trajs correctly for each world
for i in range(gpudrive_env.num_worlds):
    world_agent_indices = gpudrive_sample_batch['agents_id'][i]
    
    # Filter out negative indices (padding values)
    valid_mask = world_agent_indices >= 0  # Boolean mask of valid indices
    valid_agent_indices = world_agent_indices[valid_mask]  # Filtered tensor

    # Use tensor indexing with valid agent indices
    pred_trajs[i, valid_agent_indices, :, :2] = vbd_output[i, valid_agent_indices, :, :2] - world_means[i].view(1, 1, 2)  # pos x, y
    pred_trajs[i, valid_agent_indices, :, 3] = vbd_output[i, valid_agent_indices, :, 2]    # yaw
    pred_trajs[i, valid_agent_indices, :, 4:6] = vbd_output[i, valid_agent_indices, :, 3:5]  # vel x, y

# Now step through the simulation
gpudrive_frames = []
for t in range(env_config.episode_len-INIT_STEPS):
    gpudrive_env.step_dynamics(pred_trajs[:, :, t, :])
    fig = gpudrive_env.vis.plot_simulator_state(
        time_steps=[t],
        env_indices=[0, 1],
        zoom_radius=120,
    )[0]
    gpudrive_frames.append(img_from_fig(fig))

Diffusion: 100%|██████████| 50/50 [00:27<00:00,  1.84it/s]


In [5]:
mediapy.write_video('gpudrive/integrations/vbd/viz/train.gif', gpudrive_frames, fps=FPS, codec="gif")
print("GIF saved at gpudrive/integrations/vbd/viz/train.gif")

GIF saved at gpudrive/integrations/vbd/viz/train.gif


In [6]:
gpudrive_sample_batch['agents_id']

tensor([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]],
       dtype=torch.int32)

In [12]:
gpudrive_env.get_env_filenames()

{0: 'tfrecord-00000-of-01000_401.json', 1: 'tfrecord-00002-of-01000_321.json'}

In [7]:
print(pred_trajs.shape, vbd_output.shape)

torch.Size([2, 64, 80, 10]) torch.Size([2, 32, 80, 5])


In [8]:
world_agent_indices = gpudrive_sample_batch['agents_id']
valid_mask = world_agent_indices >= 0  # Boolean mask of valid indices
valid_agent_indices = world_agent_indices[valid_mask]  # Filtered tensor

print(world_agent_indices)
print(valid_agent_indices)

tensor([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [ 0,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]],
       dtype=torch.int32)
tensor([0, 0, 1], dtype=torch.int32)


## Plotting VBD trajectory as part of agent obs

In [9]:
def transform_to_ego_frame(trajectory: torch.Tensor, ego_pos: torch.Tensor, ego_yaw: torch.Tensor) -> torch.Tensor:
    """
    Transform trajectory from global coordinates to ego-centric frame.
    Args:
    trajectory: Shape (time_steps, 2) containing x,y coordinates in global frame
    ego_pos: Shape (2,) containing ego x,y position
    ego_yaw: Shape (1,) containing ego yaw angle in radians
    Returns:
    transformed_trajectory: Shape (time_steps, 2) in ego-centric frame
    """
    # Step 1: Translate trajectory to be relative to ego position
    translated = trajectory - ego_pos
    
    # Step 2: Rotate trajectory to align with ego orientation
    # Create rotation matrix
    cos_yaw = torch.cos(ego_yaw)
    sin_yaw = torch.sin(ego_yaw)
    rotation_matrix = torch.tensor([
        [cos_yaw, sin_yaw],
        [-sin_yaw, cos_yaw]
    ])
    
    # Apply rotation matrix to the translated trajectory
    # We need to transpose rotation_matrix for batch matrix multiplication
    transformed_trajectory = torch.matmul(translated, rotation_matrix.T)
    
    return transformed_trajectory

In [10]:
# plotting vbd trajectory as part of observation
from gpudrive.datatypes.observation import GlobalEgoState
init_state = gpudrive_env.reset()
# Get global agent observations
global_agent_obs = GlobalEgoState.from_tensor(
    abs_self_obs_tensor=gpudrive_env.sim.absolute_self_observation_tensor(),
    backend=gpudrive_env.backend,
    device=gpudrive_env.device,
)

for agent_index in valid_agent_indices:
    vbd_trajectory = gpudrive_env.vbd_trajectories[0, agent_index, :, :2]
    pos_xy = torch.tensor([global_agent_obs.pos_x[0, agent_index], global_agent_obs.pos_y[0, agent_index]])
    yaw = torch.tensor(global_agent_obs.rotation_angle[0, agent_index])

    transformed_trajectory = transform_to_ego_frame(vbd_trajectory, pos_xy, yaw)

    fig = gpudrive_env.vis.plot_agent_observation(
        agent_idx=agent_index,
        env_idx=0,
        trajectory=transformed_trajectory,
        figsize = (4, 4)
    )
    fig.savefig(f'gpudrive/integrations/vbd/viz/{agent_index}_vbd_trajectory.png', facecolor='white', transparent=False)

TypeError: 'NoneType' object is not subscriptable